# Phase 1. Foundations: Concurrency, The GIL, and Threads

In [2]:
import sys

sys.version

'3.14.0 free-threading build (main, Nov  1 2025, 00:37:47) [GCC 14.2.0]'

# 1.1 Comparing Sequential vs Concurrent

In [7]:
import time


def print_numbers(n):
    """Print numbers."""
    for i in range(n):
        time.sleep(1)
        print(f"Number: {i}")


def print_letters(string):
    """Print letters."""
    for letter in string:
        time.sleep(1)
        print(f"Letter: {letter}")


## Sequential

In [8]:
def main():
    """Run code."""
    start = time.perf_counter()

    utils.print_numbers(5)
    utils.print_letters("abcde")

    end = time.perf_counter()

    print(f"Time: {round(end - start, 2)}")


if __name__ == "__main__":
    main()


Number: 0
Number: 1
Number: 2
Number: 3
Number: 4
Letter: a
Letter: b
Letter: c
Letter: d
Letter: e
Time: 10.01


## Concurrent

In [9]:
def main():
    """Run code."""
    start = time.perf_counter()

    t1 = threading.Thread(target=utils.print_numbers, args=(5,))
    t2 = threading.Thread(target=utils.print_letters, args=("abcde",))

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    end = time.perf_counter()

    print(f"Time: {round(end - start, 2)}")


if __name__ == "__main__":
    main()


Number: 0
Letter: a
Letter: bNumber: 1

Number: 2
Letter: c
Number: 3Letter: d

Number: 4Letter: e

Time: 5.01
